In [2]:
from functions import *

2024-11-22 13:56:54.613234: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-22 13:56:54.635689: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-22 13:56:54.642680: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-22 13:56:54.660266: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-22 13:56:55.932168: W tensorflow/compiler/tf2

In [3]:
import matplotlib.pyplot as plt
import glob
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import geopandas as gpd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import glob
import time
from datetime import datetime
from collections import Counter
from PIL import Image

# Visualize one experiment

In [18]:
model='baseline_CNN'
exp = 1
spl = 2

#### Experiment details

In [19]:
df1=pd.read_excel(f'../official_results/{model}_results.xlsx')
df2=pd.read_excel(f'../official_results/old/{model}_results.xlsx')
df = pd.concat([df1, df2], ignore_index = True)
df['Normalized Variables'] = df['Variables'].apply(
    lambda x: ', '.join(sorted(x.split(', '))))


In [20]:
# Define mapping for concise names
variable_set_mapping = {
    'lst': 'LST',
    'lst, ndvi': 'LST + NDVI',
    'altitude, direction, lst, ndvi, slope': 'All Image Features',
    'discharge, lat, lon, lst, month, ndvi': 'LST + NDVI + Non-image Features',
    'altitude, direction, discharge, lat, lon, lst, month, ndvi, slope': 'Full Features'
}

# Apply mapping
df['Normalized Variables'] = df['Normalized Variables'].map(variable_set_mapping)
df

,Experiment,RMSE,Variables,Input,Split,Split_id,Optimizer,nº samples,Batch size,Epochs,Date,Time,Duration,Loss,Resolution,Normalized Variables
0,1,5.683873,lst,74 cells,stratified,3,Adam,1487,32,57 of 300,2024-11-20,10:35:18,259.58,Physics-guided,8,LST
1,2,5.095058,lst,74 cells,stratified,3,Adam,1487,32,54 of 300,2024-11-20,10:39:12,232.63,RMSE,8,LST
2,3,5.064375,lst,74 cells,stratified,3,Adam,1487,64,102 of 300,2024-11-20,10:49:56,292.51,Physics-guided,8,LST
3,4,4.963711,lst,74 cells,stratified,3,Adam,1487,64,119 of 300,2024-11-20,10:55:22,323.51,RMSE,8,LST
4,3,5.005169,lst,74 cells,stratified,3,Adam,1487,128,74 of 300,2024-11-20,10:57:50,146.31,Physics-guided,8,LST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,6,5.205161,lst,74 cells,stratified,2,Adam,1487,128,123 of 300,2024-11-19,15:47:30,265.10,Physics-guided,64,LST
190,6,4.778100,lst,74 cells,stratified,3,Adam,1487,64,108 of 300,2024-11-19,15:48:31,301.57,RMSE,64,LST
191,7,5.102246,lst,74 cells,stratified,2,Adam,1487,128,138 of 300,2024-11-19,15:52:23,290.87,RMSE,64,LST
192,6,4.888248,lst,74 cells,stratified,3,Adam,1487,128,149 of 300,2024-11-19,15:53:35,301.87,Physics-guided,64,LST


In [21]:
# Best experiment
rmse_mean = df.groupby(['Batch size', 'Resolution', 'Optimizer', 'Normalized Variables'])['RMSE'].mean().reset_index()

rmse_mean = rmse_mean.sort_values(by='RMSE')
rmse_mean.head()

,Batch size,Resolution,Optimizer,Normalized Variables,RMSE
43,128,16,Adam,LST + NDVI,4.797674
30,64,64,Adam,All Image Features,4.812507
38,128,8,Adam,LST + NDVI,4.856707
20,64,16,Adam,All Image Features,4.858441
46,128,32,Adam,Full Features,4.897588


In [22]:
df[df.RMSE == df['RMSE'].min()]

,Experiment,RMSE,Variables,Input,Split,Split_id,Optimizer,nº samples,Batch size,Epochs,Date,Time,Duration,Loss,Resolution,Normalized Variables
165,154,4.506993,"lst, ndvi, slope, direction, altitude, month, ...",74 cells,stratified,2,Adam,1487,128,67 of 300,2024-11-21,16:44:25,158.26,RMSE,64,Full Features


In [26]:
best_experiments = rmse_mean.head()
filtered_df = df.merge(best_experiments, on=['Batch size', 'Resolution', 'Optimizer', 'Normalized Variables'], how='inner')
filtered_df.sort_values(by='RMSE_x').head()

,Experiment,RMSE_x,Variables,Input,Split,Split_id,Optimizer,nº samples,Batch size,Epochs,Date,Time,Duration,Loss,Resolution,Normalized Variables,RMSE_y
14,153,4.695305,"lst, ndvi, slope, direction, altitude",74 cells,stratified,1,Adam,1487,64,78 of 300,2024-11-21,16:31:35,222.52,RMSE,64,All Image Features,4.812507
9,83,4.702397,"lst, ndvi",74 cells,stratified,1,Adam,1487,128,130 of 300,2024-11-21,11:42:38,264.94,RMSE,16,LST + NDVI,4.797674
5,55,4.762912,"lst, ndvi, slope, direction, altitude",74 cells,stratified,3,Adam,1487,64,75 of 300,2024-11-20,17:04:52,218.35,RMSE,16,All Image Features,4.858441
16,155,4.764760,"lst, ndvi, slope, direction, altitude",74 cells,stratified,3,Adam,1487,64,61 of 300,2024-11-21,16:39:04,180.70,RMSE,64,All Image Features,4.812507
6,83,4.771601,"lst, ndvi",74 cells,stratified,1,Adam,1487,128,139 of 300,2024-11-21,11:34:32,279.55,RMSE,8,LST + NDVI,4.856707


In [24]:
df[df['nº samples']==850]

,Experiment,RMSE,Variables,Input,Split,Split_id,Optimizer,nº samples,Batch size,Epochs,Date,Time,Duration,Loss,Resolution,Normalized Variables
171,172,5.343398,"lst, ndvi, slope, altitude, direction, month, ...",42 cells,stratified,1,Adam,850,128,146 of 300,2024-11-22,13:41:28,208.33,RMSE,64,Full Features
172,173,6.396033,"lst, ndvi, slope, altitude, direction, month, ...",42 cells,stratified,2,Adam,850,128,80 of 300,2024-11-22,13:43:19,110.02,RMSE,64,Full Features
173,174,13.978865,"lst, ndvi, slope, altitude, direction, month, ...",42 cells,stratified,3,Adam,850,128,31 of 300,2024-11-22,13:44:04,43.38,RMSE,64,Full Features
174,175,7.616661,"lst, ndvi, month, discharge, lat, lon",42 cells,stratified,1,Adam,850,16,43 of 300,2024-11-22,14:12:30,234.15,RMSE,64,LST + NDVI + Non-image Features
175,175,5.878467,"lst, month, discharge, lat, lon",42 cells,stratified,1,Adam,850,16,51 of 300,2024-11-22,14:13:59,275.99,RMSE,64,NaN


#### Learning curve

In [ ]:
image_path = f'../official_results/learning_curves/{model}_exp_{exp}_split_{spl}.png'
image = Image.open(image_path)
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.axis('off')
plt.show()


#### Error log

In [ ]:
errors_df = pd.read_csv(f'../official_results/error_logs/{model}_exp_{exp}_split_{spl}.csv')

In [ ]:
errors_df_filt = errors_df[errors_df['epoch'] <= 100]

In [ ]:
plt.figure(figsize=(12, 6))
sns.scatterplot(data=errors_df_filt, x="month", y="error", hue="epoch", palette="viridis", alpha=0.7)
plt.xticks(ticks=sorted(errors_df_filt["month"].unique()))
plt.title("Error depending on the months")
plt.xlabel("Month")
plt.ylabel('Error')
plt.show()

In [ ]:
epochs = len(errors_df_filt.epoch.unique())
pivot_table = errors_df_filt.pivot_table(values="error", index="epoch", columns="month", aggfunc="mean")
plt.figure(figsize=(12, 6))
sns.heatmap(pivot_table, cmap="YlGnBu", cbar_kws={'label': 'Average RMSE'})
if epochs == 10:
    ticks = range(epochs + 1)
    labelss = [str(i) for i in range(epochs + 1)]
else:
    ticks = [i for i in range(10, epochs + 1, 10)]
    labelss = [str(i) for i in range(10, epochs + 1, 10)]
plt.yticks(ticks=ticks,labels=labelss)
plt.title("Heatmap of average RMSE by epoch and month")
plt.xlabel("Month")
plt.ylabel("Epoch")
plt.savefig(f"../plots/season_learning_exp_{exp}_split_{spl}.png")
plt.show()

# Analysis of all baseline experiments

### Compare RMSE for all baseline experiments

In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(df.index, df['RMSE'], color='skyblue', label="RMSE")
plt.axhline(y=df['RMSE'].min(), color='red', linestyle='--', label="Lowest RMSE")
plt.xlabel('Experiment')
plt.ylabel('RMSE')
plt.title('RMSE Across Experiments')
plt.legend()
plt.show()

### Best batch and resolution combination

In [ ]:
mean_rmse = df.groupby(["Resolution", "Batch size", "Loss"])["RMSE"].mean().reset_index()

# Identificar la combinación con el menor RMSE
best_combination = mean_rmse.loc[mean_rmse["RMSE"].idxmin()]
print(best_combination)
best_experiments = df[
    (df["Resolution"] == best_combination["Resolution"]) &
    (df["Batch size"] == best_combination["Batch size"]) &
    (df["Loss"] == best_combination["Loss"])
]

# Mostrar todas las variables para estas filas
print("Best Experiments Details:")
best_experiments

# Training parameters experimentation

## Image resolution and batch size analysis

### Best combination (splits mean) w/lst

In [ ]:
df_lst = df[df.Variables=='lst']
mean_rmse = df_lst.groupby(["Resolution", "Batch size", "Loss"])["RMSE"].mean().reset_index()

# Identificar la combinación con el menor RMSE
best_combination = mean_rmse.loc[mean_rmse["RMSE"].idxmin()]
print(best_combination)
best_experiments = df_lst[
    (df["Resolution"] == best_combination["Resolution"]) &
    (df["Batch size"] == best_combination["Batch size"]) &
    (df["Loss"] == best_combination["Loss"])
]

# Mostrar todas las variables para estas filas
print("Best Experiments Details:")
best_experiments

In [ ]:
# Ordenar las combinaciones por RMSE promedio
mean_rmse_sorted = mean_rmse.sort_values("RMSE")

print("Top Combinations by Average RMSE:")
print(mean_rmse_sorted.head(10))  # Ver las 10 mejores combinaciones


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

custom_palette = {
    32: "#66c2a4",  # Verde
    64: "#8c6bb1",  # Lila
    128: "#d73027"  # Rojo
}
# Crear un gráfico para los datos filtrados
plt.figure(figsize=(10, 6))
sns.lineplot(
    data=mean_rmse,
    x="Resolution",
    y="RMSE",
    hue="Batch size",
    style="Loss",  # Diferenciar por tipo de Loss
    markers=True,
    palette=custom_palette  # Paleta de colores
)

resolutions_used = sorted(mean_rmse["Resolution"].unique())  # Obtener resoluciones únicas para este Split_id
plt.xticks(resolutions_used, labels=[str(res) for res in resolutions_used], fontsize=12)


# Configurar título y ejes
plt.title(f"RMSE vs. Resolution", fontsize=14)
plt.xlabel("Image Resolution (pixels)", fontsize=12)
plt.ylabel("RMSE", fontsize=12)
plt.legend(title="Batch Size and Loss Type", fontsize=10)
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()

# Mostrar el gráfico
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Filtrar por tipo de Loss (por ejemplo, "Physics-guided")
physics_mean_rmse = mean_rmse[mean_rmse["Loss"] == "Physics-guided"].pivot(
    index="Resolution", columns="Batch size", values="RMSE"
)

plt.figure(figsize=(8, 6))
sns.heatmap(physics_mean_rmse, annot=True, fmt=".2f", cmap="coolwarm", cbar_kws={'label': 'Average RMSE'})
plt.title("Average RMSE by Resolution and Batch Size (Physics-guided)")
plt.xlabel("Batch Size")
plt.ylabel("Resolution (pixels)")
plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Filtrar por tipo de Loss (por ejemplo, "Physics-guided")
physics_mean_rmse = mean_rmse[mean_rmse["Loss"] == "RMSE"].pivot(
    index="Resolution", columns="Batch size", values="RMSE"
)

plt.figure(figsize=(8, 6))
sns.heatmap(physics_mean_rmse, annot=True, fmt=".2f", cmap="coolwarm", cbar_kws={'label': 'Average RMSE'})
plt.title("Average RMSE by Resolution and Batch Size (RMSE)")
plt.xlabel("Batch Size")
plt.ylabel("Resolution (pixels)")
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Gráfico de líneas para cada batch size
plt.figure(figsize=(10, 6))
sns.lineplot(data=mean_rmse, x="Resolution", y="RMSE", hue="Batch size", marker="o", palette="viridis")
plt.title("RMSE vs. Resolution for Different Batch Sizes", fontsize=14)
plt.xlabel("Image Resolution (pixels)", fontsize=12)
plt.ylabel("RMSE", fontsize=12)
plt.legend(title="Batch Size", fontsize=10)
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

## Optimizer analysis

In [ ]:
best_results = df_lst[(df_lst.Resolution==32)&(df['Batch size']==128)&(df['Loss']=='RMSE')]
best_results

In [ ]:
rmse_mean = best_results.groupby('Optimizer')['RMSE'].mean().reset_index()

rmse_mean = rmse_mean.sort_values(by='RMSE')

print(rmse_mean)

# Variables experimentation

In [ ]:
df_vars = df[(df.Optimizer=='Adam')&(df.Loss=='RMSE')]
df_vars

In [ ]:
# Agrupar por las variables normalizadas y calcular la media del RMSE
variables_rmse = df_vars.groupby('Normalized Variables')['RMSE'].mean().sort_values()

# Plot mean RMSE for each variable set
plt.figure(figsize=(12, 6))
variables_rmse.plot(kind='bar', color='green')
plt.xlabel('Variable Sets')
plt.ylabel('Mean RMSE')
plt.title('Mean RMSE by Variable Sets')
plt.xticks(rotation=45, ha='right')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Crear una nueva columna combinada para Resolution, Batch Size y Variables
df['Resolution-BatchSize-Variables'] = (
    df['Resolution'].astype(str) + 'x' + df['Batch size'].astype(str) + ' - ' + df['Normalized Variables']
)

# Agrupar por la combinación y calcular la media del RMSE
combination_rmse = df.groupby('Resolution-BatchSize-Variables')['RMSE'].mean().sort_values()

# Crear el gráfico
plt.figure(figsize=(12, 6))
combination_rmse.plot(kind='bar', color='purple')
plt.xlabel('Resolution, Batch Size, and Variables')
plt.ylabel('Mean RMSE')
plt.title('Mean RMSE by Resolution, Batch Size, and Variables')
plt.xticks(rotation=90, ha='right')
plt.show()


In [ ]:
# RMSE promedio por resolución
rmse_by_resolution = df.groupby('Resolution')['RMSE'].mean().sort_values()
print("RMSE by Resolution:")
print(rmse_by_resolution)

# RMSE promedio por batch size
rmse_by_batch = df.groupby('Batch size')['RMSE'].mean().sort_values()
print("\nRMSE by Batch Size:")
print(rmse_by_batch)

# RMSE promedio por variables
rmse_by_variables = df.groupby('Variables')['RMSE'].mean().sort_values()
print("\nRMSE by Variables:")
print(rmse_by_variables)

# RMSE promedio por la combinación Resolution-BatchSize-Variables
rmse_by_combination = df.groupby('Resolution-BatchSize-Variables')['RMSE'].mean().sort_values()
print("\nRMSE by Combination:")
print(rmse_by_combination)


In [ ]:
# RMSE promedio por resolución y batch size
rmse_res_batch = df.groupby(['Resolution', 'Batch size'])['RMSE'].mean().unstack()

# Visualizar como heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(rmse_res_batch, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Mean RMSE by Resolution and Batch Size")
plt.xlabel("Batch Size")
plt.ylabel("Resolution")
plt.show()


In [ ]:
# RMSE promedio por resolución y variables
rmse_res_vars = df.groupby(['Resolution', 'Normalized Variables'])['RMSE'].mean().unstack()

# Visualizar como heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(rmse_res_vars, annot=True, cmap='viridis', fmt=".2f")
plt.title("Mean RMSE by Resolution and Variables")
plt.xlabel("Variables")
plt.ylabel("Resolution")
plt.xticks(rotation=45, ha='right')
plt.show()


In [ ]:
# RMSE promedio por resolución y variables
rmse_res_vars = df.groupby(['Batch size', 'Normalized Variables'])['RMSE'].mean().unstack()

# Visualizar como heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(rmse_res_vars, annot=True, cmap='viridis', fmt=".2f")
plt.title("Mean RMSE by Resolution and Variables")
plt.xlabel("Variables")
plt.ylabel("Batch size")
plt.xticks(rotation=45, ha='right')
plt.show()


In [ ]:
df[(df['Normalized Variables']=='All Image Features')&(df.Resolution==32)&(df['Batch size']==128)]


In [ ]:
correlations = df[['Resolution', 'Batch size', 'RMSE']].corr()
print("Correlations:")
print(correlations)

# Heatmap de correlaciones
sns.heatmap(correlations, annot=True, cmap='coolwarm')
plt.title("Correlation Heatmap")
plt.show()


# CNN_2 analysis

In [11]:
model= 'CNN_2'
df=pd.read_excel(f'../official_results/{model}_results.xlsx')
df

,Experiment,RMSE,Variables,Input,Split,Split_id,Optimizer,nº samples,Batch size,Epochs,Date,Time,Duration,Loss,Resolution
0,1,10.308114,"lst, ndvi, slope, direction, altitude, month, ...",42 cells,stratified,1,Adam,850,128,38 of 300,2024-11-22,13:37:44,59.00,RMSE,64
1,2,8.166103,"lst, ndvi, slope, direction, altitude, month, ...",42 cells,stratified,2,Adam,850,128,36 of 300,2024-11-22,13:38:38,52.41,RMSE,64
2,3,8.051337,"lst, ndvi, slope, direction, altitude, month, ...",42 cells,stratified,3,Adam,850,128,39 of 300,2024-11-22,13:39:38,58.98,RMSE,64
3,3,4.928916,"lst, ndvi, altitude, direction, slope, month, ...",74 cells,stratified,1,Adam,1487,64,55 of 300,2024-11-22,13:42:37,196.52,RMSE,64
4,4,5.037980,"lst, ndvi, altitude, direction, slope, month, ...",74 cells,stratified,2,Adam,1487,64,109 of 300,2024-11-22,13:48:49,370.03,RMSE,64
5,5,5.055923,"lst, ndvi, altitude, direction, slope, month, ...",74 cells,stratified,3,Adam,1487,64,53 of 300,2024-11-22,13:51:52,180.21,RMSE,64
6,3,5.031001,"lst, ndvi, altitude, direction, slope, month, ...",74 cells,stratified,1,Adam,1487,128,82 of 300,2024-11-22,13:55:05,190.05,RMSE,64
7,4,4.710206,"lst, ndvi, altitude, direction, slope, month, ...",74 cells,stratified,2,Adam,1487,128,37 of 300,2024-11-22,13:56:35,88.02,RMSE,64
8,5,4.744955,"lst, ndvi, altitude, direction, slope, month, ...",74 cells,stratified,3,Adam,1487,128,41 of 300,2024-11-22,13:58:15,97.38,RMSE,64


# Resnet analyis

In [16]:
model= 'Resnet'
df=pd.read_excel(f'../official_results/{model}_results.xlsx')
df

,Experiment,RMSE,Variables,Input,Split,Split_id,Optimizer,nº samples,Batch size,Epochs,Date,Time,Duration,Loss,Resolution
0,1,4.713529,"lst, ndvi, slope, direction, altitude, month, ...",74 cells,stratified,1,Adam,1487,128,56 of 300,2024-11-22,13:58:40,258.65,RMSE,64
1,2,4.897418,"lst, ndvi, slope, direction, altitude, month, ...",74 cells,stratified,2,Adam,1487,128,65 of 300,2024-11-22,14:03:39,296.52,RMSE,64
2,3,4.762808,"lst, ndvi, slope, direction, altitude, month, ...",74 cells,stratified,3,Adam,1487,128,67 of 300,2024-11-22,14:08:47,305.53,RMSE,64
